In [4]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm.auto import tqdm
from rdkit.Chem.Draw import MolsToGridImage
import seaborn as sns
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [16]:
#dfMix = pd.read_csv("../../Met_Files/Acetylcholinesterase_CHEMBL220.csv")
#targetName = "Acetyl"

dfMix = pd.read_csv("../../Met_Files/Monoamine_oxidase_A_CHEMBL1951.csv")
targetName = "Mono"

#dfMix = pd.read_csv("../../Met_Files/Protein-tyrosine_phosphatase_1B_CHEMBL335.csv")
#targetName = "Tyro"

In [ ]:
def morganHelper(smiles, radius=2, n_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    return list(fp)
dfMor = pd.DataFrame(data = [], columns = ['MorganFingerprint'])
dfMor['MorganFingerprint'] = dfMix['SMILES'].apply(morganHelper)
dfMor = dfMor.dropna(subset=['MorganFingerprint'])
dfMorgan = pd.DataFrame(dfMor['MorganFingerprint'].tolist())

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

origData = dfMorgan
allData = origData

allTargets = dfMix['pIC50']

In [ ]:
from sklearn.preprocessing import StandardScaler
allDataVals = StandardScaler().fit_transform(allData.values)
allTargetVals = allTargets.values

In [ ]:
#from sklearn.decomposition import PCA
#pca = PCA(n_components = 50)
#allDataComp = pca.fit_transform(allDataVals)

In [ ]:
tsne = TSNE(n_components = 2)
tsne_crds = tsne.fit_transform(allDataVals)

In [ ]:
tsne_df = pd.DataFrame(tsne_crds,columns=["X","Y"])
tsne_df['is_metabolite'] = dfMix['natural_product']
tsne_df.head()

In [ ]:
ax = sns.scatterplot(data=tsne_df.query("is_metabolite == False"),x="X",y="Y",color='lightblue', label = "Non-Natural Product")
ax = sns.scatterplot(data=tsne_df.query("is_metabolite == True"),x="X",y="Y",color='red', label = "Natural Product")
plt.title(f"{targetName}-tSNE")
plt.savefig(f"{targetName}-tSNE")
plt.legend()